## Silver Data Transformation

In [0]:
df_nf_titles = spark.read.format("delta")\
            .option("header", "true")\
              .option("inferSchema", "true")\
                .load("abfss://bronze@storagenetflixashu.dfs.core.windows.net/netflix_titles")

In [0]:
df_nf_titles.display()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_nf_titles = df_nf_titles.fillna({"duration_minutes": 0, "duration_seasons": 1})

In [0]:
df_nf_titles.display()

In [0]:
df_nf_titles  = df_nf_titles.withColumn("duration_minutes", df_nf_titles["duration_minutes"].cast(IntegerType()))\
                                .withColumn("duration_seasons", df_nf_titles["duration_seasons"].cast(IntegerType()))

In [0]:
df_nf_titles.display()

In [0]:
df_nf_titles = df_nf_titles.withColumn("shortTitle", split(col("title"),":")[0])

In [0]:
df_nf_titles.display()

In [0]:
df_nf_titles = df_nf_titles.withColumn("rating", split(col("rating"),"-")[0])

In [0]:
df_nf_titles.display()

In [0]:
df_nf_titles = df_nf_titles.withColumn("type_flag", when(col("type") == 'Movie', 1)\
                                                                   .when(col("type") == 'TV Show', 2)\
                                                                       .otherwise(0))

In [0]:
df_nf_titles.display()

In [0]:
from pyspark.sql.window import Window

In [0]:
df_nf_titles = df_nf_titles.withColumn("duration_rank", dense_rank().over(Window.orderBy(col("duration_minutes").desc())))

In [0]:
df_nf_titles.display()

In [0]:
df_nf_titles_agg = df_nf_titles.groupBy('type').agg(count('*').alias('otal_count'))
df_nf_titles_agg.display()

In [0]:
df_nf_titles.write.format('delta')\
                .mode('overwrite')\
                    .option('path','abfss://silver@storagenetflixashu.dfs.core.windows.net/netflix_titles')\
                        .save()